In [ ]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('DF').getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql.types import *


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=2c657743e381adb48ba2d77a8fa1efe1df8ba1c01d1282f93ab61e0d9a4491a4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
df =spark.read.csv('/content/indian_food.csv',header=True,inferSchema=True)

In [ ]:
df.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [ ]:
dfCleaned = df.withColumn("state", when(col("state") == "-1", None).otherwise(col("state"))) \
    .withColumn("region", when(col("region") == "-1", None).otherwise(col("region")))

In [ ]:
dfCleaned.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [ ]:
# 1. Find out how many unique dishes are present
uniqueDishesCount = dfCleaned.select("name").distinct().count()
uniqueDishesCount

255

In [ ]:
# 2. Which state has more dishes
stateWithMoreDishes = dfCleaned.groupBy("state").count().orderBy(desc("count")).first()
stateWithMoreDishes

Row(state='Gujarat', count=35)

In [ ]:
# 3. How many dishes from the state Karnataka
dishesFromKarnataka = dfCleaned.filter(col("state") == "Karnataka").count()
dishesFromKarnataka

6

In [ ]:
# 4. List the number of unique regions
uniqueRegions = dfCleaned.select("region").distinct().count()
uniqueRegions

7

In [ ]:
# 5. Count the number of dishes from each region
dishesPerRegion = dfCleaned.groupBy("region").count().orderBy("region")
dishesPerRegion

DataFrame[region: string, count: bigint]

In [ ]:
uniqueFlavorProfiles = dfCleaned.select("flavor_profile").distinct().collect()
uniqueCourses = dfCleaned.select("course").distinct().collect()
uniqueFlavorProfiles


[Row(flavor_profile='-1'),
 Row(flavor_profile='spicy'),
 Row(flavor_profile='sweet'),
 Row(flavor_profile='sour'),
 Row(flavor_profile='bitter')]

In [ ]:
uniqueCourses

[Row(course='starter'),
 Row(course='dessert'),
 Row(course='snack'),
 Row(course='main course')]

In [ ]:
stateWithMoreMainCourse = dfCleaned.filter(col("course") == "main course").groupBy("state").count().orderBy(desc("count")).first()
stateWithMoreMainCourse

Row(state='Punjab', count=28)

In [ ]:
totalDishes = dfCleaned.count()
dishesPercentagePerRegion = dishesPerRegion.withColumn("percentage", (col("count") / totalDishes) * 100)
dishesPercentagePerRegion.show()

+----------+-----+------------------+
|    region|count|        percentage|
+----------+-----+------------------+
|      NULL|   14| 5.490196078431373|
|   Central|    3|1.1764705882352942|
|      East|   31|12.156862745098039|
|     North|   49|19.215686274509807|
|North East|   25| 9.803921568627452|
|     South|   59|23.137254901960784|
|      West|   74| 29.01960784313726|
+----------+-----+------------------+

